# Shows how to Use the Data Alan Created and Use Utility Functions

In [1]:
import pickle
from collections import namedtuple
import pandas as pd
import numpy as np
from importlib import reload
import sys

# bench_util module is in the parent directory, because it will be part of
# the production code.  Add the parent directory to the Path so Python can
# import from there
sys.path.insert(0, '../')   
import bench_util

## Unpickle the Dataframe for Preprocessed Utility Data & Make Utility Function Object

In [6]:
# Unpickle the pre-processed DataFrame
df = pickle.load(open('dfu3.pkl', 'rb'))

# Unpickle the raw utility bill DataFrame, which is needed below to make
# the utility function object.
df_raw = pickle.load(open('df_raw.pkl', 'rb'))

df.head()  # the processed data

,site_id,service_type,cal_year,cal_mo,item_desc,units,cost,days_served,usage,fiscal_year,fiscal_mo,mmbtu
0,03,Electricity,2010,7,Electricity charge,-,24.137931,35.0,0.000000,2011,1,NaN
1,03,Electricity,2010,7,Electricity charge,kWh,1304.274655,9.5,6656.551724,2011,1,22.712154
2,03,Electricity,2010,8,Electricity charge,-,39.498433,62.0,0.000000,2011,2,NaN
3,03,Electricity,2010,8,Electricity charge,kWh,4940.646254,31.0,26822.236155,2011,2,91.517470
4,03,Electricity,2010,9,Electricity charge,-,16.363636,27.0,0.000000,2011,3,NaN


In [13]:
# this is only needed to update any code changes I may have made
# since last importing the module above.
reload(bench_util)     

# Make an object that has the various utility functions.
# The object needs access to the raw utility bill DataFrame and the spreadsheet
# containing other application data.
ut = bench_util.Util(df_raw, '../data/Other_Building_Data.xlsx')

## DataFrame for "Utility Cost Overview" Report Page

In [4]:
site = 'ANSBG1'
# From the main DataFrame, get only the rows for this site, and only get
# the needed columns for this analysis
df1 = df.query('site_id == @site')[['service_type', 'fiscal_year', 'fiscal_mo', 'cost']]
df1.head()

,service_type,fiscal_year,fiscal_mo,cost
17200,Electricity,2011,1,602.300
17201,Electricity,2011,1,0.000
17202,Electricity,2011,1,699.360
17203,Electricity,2011,2,1212.035
17204,Electricity,2011,2,0.000


In [5]:
df2 = pd.pivot_table(
    df1, 
    values='cost', 
    index=['fiscal_year'], 
    columns=['service_type'],
    aggfunc=np.sum
)
df2

service_type,Electricity,Natural Gas,Sewer,Water
fiscal_year,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667
2012,41994.697841,94490.242371,5484.028148,5090.420833
2013,36961.451595,79970.329194,5169.324500,4783.854500
2014,35843.822155,48967.142581,4840.116833,4375.768167
2015,38017.419167,47128.677258,4593.016167,4156.942333
2016,29639.448485,45647.201628,6108.227500,5113.362500
2017,21849.234848,26682.420469,3326.440000,2875.330000


In [6]:
# Add in columns for the missing services
missing_services = bench_util.missing_services(df2.columns)
bench_util.add_columns(df2, missing_services)
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse
fiscal_year,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0


In [7]:
# Add a Total column that sums the other columns
df2['Total'] = df2.sum(axis=1)
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse,Total
fiscal_year,,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0,153849.180928
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0,147059.389193
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0,126884.959788
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0,94026.849736
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0,93896.054925
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0,86508.240113
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0,54733.425317


In [8]:
# Add a percent change column
df2['pct_change'] = df2.Total.pct_change()
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse,Total,pct_change
fiscal_year,,,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0,153849.180928,NaN
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0,147059.389193,-0.044133
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0,126884.959788,-0.137186
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0,94026.849736,-0.258960
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0,93896.054925,-0.001391
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0,86508.240113,-0.078681
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0,54733.425317,-0.367304


In [9]:
# Add in degree days
months_present = bench_util.months_present(df1)
deg_days = ut.degree_days_yearly(months_present, site)
df2['hdd'] = deg_days
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse,Total,pct_change,hdd
fiscal_year,,,,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0,153849.180928,NaN,13802
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0,147059.389193,-0.044133,12393
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0,126884.959788,-0.137186,12171
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0,94026.849736,-0.258960,13802
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0,93896.054925,-0.001391,12393
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0,86508.240113,-0.078681,12171
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0,54733.425317,-0.367304,12356


In [10]:
# Add in a column to show the numbers of months present for each year
# This will help to identify partial years.
mo_count = bench_util.month_count(months_present)
df2['month_count'] = mo_count
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse,Total,pct_change,hdd,month_count
fiscal_year,,,,,,,,,,,
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0,153849.180928,NaN,13802,12
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0,147059.389193,-0.044133,12393,12
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0,126884.959788,-0.137186,12171,12
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0,94026.849736,-0.258960,13802,12
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0,93896.054925,-0.001391,12393,12
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0,86508.240113,-0.078681,12171,12
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0,54733.425317,-0.367304,12356,9


In [11]:
# Reverse the DataFrame
df2.sort_index(ascending=False, inplace=True)
df2

service_type,Electricity,Natural Gas,Sewer,Water,Steam,Oil #1,Refuse,Total,pct_change,hdd,month_count
fiscal_year,,,,,,,,,,,
2017,21849.234848,26682.420469,3326.440000,2875.330000,0.0,0.0,0.0,54733.425317,-0.367304,12356,9
2016,29639.448485,45647.201628,6108.227500,5113.362500,0.0,0.0,0.0,86508.240113,-0.078681,12171,12
2015,38017.419167,47128.677258,4593.016167,4156.942333,0.0,0.0,0.0,93896.054925,-0.001391,12393,12
2014,35843.822155,48967.142581,4840.116833,4375.768167,0.0,0.0,0.0,94026.849736,-0.258960,13802,12
2013,36961.451595,79970.329194,5169.324500,4783.854500,0.0,0.0,0.0,126884.959788,-0.137186,12171,12
2012,41994.697841,94490.242371,5484.028148,5090.420833,0.0,0.0,0.0,147059.389193,-0.044133,12393,12
2011,33894.895909,110858.416500,4716.576852,4379.291667,0.0,0.0,0.0,153849.180928,NaN,13802,12


In [12]:
# Export the rows of the DataFrame into a list of dictionaries for use
# in the template.  Rename columns to match template variables.
row_dicts = bench_util.df_to_dictionaries(df2, change_names={'Natural Gas': 'natural_gas', 'Oil #1': 'fuel_oil'})
row_dicts

[{'Electricity': 21849.234848484841,
  'Refuse': 0.0,
  'Sewer': 3326.4400000000001,
  'Steam': 0.0,
  'Total': 54733.42531723484,
  'Water': 2875.3299999999995,
  'fiscal_year': 2017,
  'fuel_oil': 0.0,
  'hdd': 12356.0,
  'month_count': 9.0,
  'natural_gas': 26682.420468749999,
  'pct_change': -0.36730390947936598},
 {'Electricity': 29639.448484848483,
  'Refuse': 0.0,
  'Sewer': 6108.2274999999991,
  'Steam': 0.0,
  'Total': 86508.240112872678,
  'Water': 5113.3625000000011,
  'fiscal_year': 2016,
  'fuel_oil': 0.0,
  'hdd': 12171.0,
  'month_count': 12.0,
  'natural_gas': 45647.201628024202,
  'pct_change': -0.078680779696023495},
 {'Electricity': 38017.419166666674,
  'Refuse': 0.0,
  'Sewer': 4593.0161666666672,
  'Steam': 0.0,
  'Total': 93896.054924731172,
  'Water': 4156.9423333333343,
  'fiscal_year': 2015,
  'fuel_oil': 0.0,
  'hdd': 12393.0,
  'month_count': 12.0,
  'natural_gas': 47128.677258064497,
  'pct_change': -0.0013910368310102728},
 {'Electricity': 35843.8221551724

In [13]:
# Put results into the final report dictionary that will be passed to the Template.
# This assumes you have created a couple graphs with the file names shown below.
pg3 = dict(rows=row_dicts, 
           g1='output/images/{}_pg2_g1.png'.format(site), 
           g2='output/images/{}_pg2_g2.png'.format(site)
      )
pg3

{'g1': 'output/images/ANSBG1_pg2_g1.png',
 'g2': 'output/images/ANSBG1_pg2_g2.png',
 'rows': [{'Electricity': 21849.234848484841,
   'Refuse': 0.0,
   'Sewer': 3326.4400000000001,
   'Steam': 0.0,
   'Total': 54733.42531723484,
   'Water': 2875.3299999999995,
   'fiscal_year': 2017,
   'fuel_oil': 0.0,
   'hdd': 12356.0,
   'month_count': 9.0,
   'natural_gas': 26682.420468749999,
   'pct_change': -0.36730390947936598},
  {'Electricity': 29639.448484848483,
   'Refuse': 0.0,
   'Sewer': 6108.2274999999991,
   'Steam': 0.0,
   'Total': 86508.240112872678,
   'Water': 5113.3625000000011,
   'fiscal_year': 2016,
   'fuel_oil': 0.0,
   'hdd': 12171.0,
   'month_count': 12.0,
   'natural_gas': 45647.201628024202,
   'pct_change': -0.078680779696023495},
  {'Electricity': 38017.419166666674,
   'Refuse': 0.0,
   'Sewer': 4593.0161666666672,
   'Steam': 0.0,
   'Total': 93896.054924731172,
   'Water': 4156.9423333333343,
   'fiscal_year': 2015,
   'fuel_oil': 0.0,
   'hdd': 12393.0,
   'month

## Fiscal Month Abbreviations

In [14]:
bench_util.mo_list

['Jul',
 'Aug',
 'Sep',
 'Oct',
 'Nov',
 'Dec',
 'Jan',
 'Feb',
 'Mar',
 'Apr',
 'May',
 'Jun']

In [15]:
bench_util.mo_dict

{1: 'Jul',
 2: 'Aug',
 3: 'Sep',
 4: 'Oct',
 5: 'Nov',
 6: 'Dec',
 7: 'Jan',
 8: 'Feb',
 9: 'Mar',
 10: 'Apr',
 11: 'May',
 12: 'Jun'}

## Get Additional Information on a Building

In [16]:
# Request the building using the Site ID
bldg = ut.building_info('ANSBG1')
bldg

BldgInfo(site_name='FNSB-Animal Control', site_category='Other', address='2408 Davis Road', city='Fairbanks', primary_func='Animal Shelter', year_built=1993.0, sq_ft=10123, onsite_gen=nan, dd_site='PAFA', source_elec='Golden Valley Electric', source_oil='', source_nat_gas='Fairbanks Natural Gas', source_steam='', source_water='Golden Heart Utilities', source_sewer='Golden Heart Utilities', source_refuse='', acct_elec='31850', acct_oil='', acct_nat_gas='10282  (2408 DAVIS)', acct_steam='', acct_water='1311001100', acct_sewer='1311001100', acct_refuse='')

In [17]:
# Shows some fields of info about the building
bldg.site_name, bldg.sq_ft, bldg.dd_site

('FNSB-Animal Control', 10123, 'PAFA')

In [14]:
# A DataFrame of all the building information
ut.building_info_df()

,acct_elec,acct_nat_gas,acct_oil,acct_refuse,acct_sewer,acct_steam,acct_water,address,city,dd_site,...,site_name,source_elec,source_nat_gas,source_oil,source_refuse,source_sewer,source_steam,source_water,sq_ft,year_built
site_id,,,,,,,,,,,,,,,,,,,,,
03,172230-5,10283 - DENALI,55010001,AW23-DNL03,1998005610,,1998005610,NaN,NaN,PAFA,...,SD-Denali Elementary,Golden Valley Electric,Fairbanks Natural Gas,Sourdough Fuel (Petro Star),Alaska Waste,Golden Heart Utilities,,Golden Heart Utilities,10123,NaN
04,172995-3,10283 - NORDALE,55017001,AW16-NDL04,2440271010,,2440271010,NaN,NaN,PAFA,...,SD-Nordale Elementary,Golden Valley Electric,Fairbanks Natural Gas,Sourdough Fuel (Petro Star),Alaska Waste,Golden Heart Utilities,,Golden Heart Utilities,10123,NaN
05,"104270-4, 104271-2, 321383, 357683, 566057",,55016001,AW17-LTH05,1918027500,"08-112-0, 08-113-0",1918027500,NaN,NaN,PAFA,...,SD-Lathrop High,Golden Valley Electric,,Sourdough Fuel (Petro Star),Alaska Waste,Golden Heart Utilities,Aurora Energy,Golden Heart Utilities,10123,NaN
06,112356-1,10283 - HUNTER,55012001,AW21-HNT06,1995007000,,1995007000,NaN,NaN,PAFA,...,SD-Hunter Elementary,Golden Valley Electric,Fairbanks Natural Gas,Sourdough Fuel (Petro Star),Alaska Waste,Golden Heart Utilities,,Golden Heart Utilities,10123,NaN
07,76471-2,10283-U-PARK,55028001,AW04-UPK07,4012700710,,4012700710,NaN,NaN,PAFA,...,SD-University Park Elementary,Golden Valley Electric,Fairbanks Natural Gas,Sourdough Fuel (Petro Star),Alaska Waste,College Utilities,,College Utilities,10123,NaN
08,"104487-4, 162846-0",,55006001,AW25-BNT08,1919073100,11-290-1,1919073100,NaN,NaN,PAFA,...,SD-Barnette Magnet,Golden Valley Electric,,Sourdough Fuel (Petro Star),Alaska Waste,Golden Heart Utilities,Aurora Energy,Golden Heart Utilities,10123,NaN
09,62853-7,10283 - JOY,55014001,AW19-JOY09,2610703100,,2610703100,NaN,NaN,PAFA,...,SD-Joy Elementary,Golden Valley Electric,Fairbanks Natural Gas,Sourdough Fuel (Petro Star),Alaska Waste,Golden Heart Utilities,,Golden Heart Utilities,10123,NaN
10,5502-0,,55024001,153175-0033,,,Salcha,NaN,NaN,PAFA,...,SD-Salcha Elementary,Golden Valley Electric,,Sourdough Fuel (Petro Star),Alaska Waste,,,Pioneer Wells Inc,10123,NaN
104,220060-8,10283 - NUTRITION SRVS,55002001,AW00-ENSC,2909001600,11-188-1,2909001600,NaN,NaN,PAFA,...,SD-Nutrition Services Center,Golden Valley Electric,Fairbanks Natural Gas,Sourdough Fuel (Petro Star),Alaska Waste,Golden Heart Utilities,Aurora Energy,Golden Heart Utilities,10123,NaN


In [18]:
# Get a list of all Site IDs, sorted alphabetically
ut.all_sites()

['03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '104',
 '11',
 '12',
 '13',
 '14',
 '15',
 '15A',
 '15B',
 '16',
 '20',
 '22',
 '23',
 '27',
 '28',
 '29',
 '34',
 '36',
 '37',
 '38',
 '39',
 '40',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '49',
 '76',
 'ADLER',
 'ANSBG1',
 'ASLC18',
 'ASLC21',
 'ASLC47',
 'ASLC52',
 'ASLCHU',
 'ASLCV1',
 'ASLELC1',
 'ASLELC2',
 'ASLGDM',
 'ASLGP2',
 'ASLHIS',
 'ASLPIH',
 'ASLPL1',
 'ASLSEA',
 'ASLSQD',
 'ASLTVR',
 'BALHHW',
 'BAOBG1',
 'BAP001',
 'BENBG1',
 'BHPBHG',
 'BHPBHL',
 'BHPBHW',
 'BHPCCS',
 'BHPSKI1',
 'BHPSKI2',
 'BHPSKI3',
 'BHPSKI4',
 'CACBG1',
 'CBS001',
 'CEC',
 'CLX001',
 'CLX002',
 'CLX003',
 'CLX004',
 'CLXES1',
 'CLXGP2',
 'CLXSM1',
 'CLXSO1',
 'CRB001',
 'CSP001',
 'DIPMP1',
 'DOGPRK',
 'Emer S T',
 'GF001',
 'GFP001',
 'GRP001',
 'GRPLFT',
 'GSWNP',
 'HEMBG1',
 'HEZ001',
 'HSPSWP',
 'KEP001',
 'KIP001',
 'KWP001',
 'LEABG1',
 'LF001',
 'MF001',
 'MNPPRK',
 'MSLL001',
 'MSP001',
 'MSRSWP',
 'MSWBG1',
 'MSWBG2',
 'MSWWAR',


In [35]:
ut.site_categories_and_buildings()

[{'category': 'Other',
  'sites': [('FNSB-Adler School building', 'ADLER'),
   ('FNSB-Animal Control', 'ANSBG1'),
   ('FNSB-Bernice Aldridge Park', 'BAP001'),
   ('FNSB-Big Dipper', 'DIPMP1'),
   ('FNSB-Birch Hill Garage', 'BHPBHG'),
   ('FNSB-Birch Hill Ski Building', 'BHPCCS'),
   ('FNSB-Birch Hill Ski Trail #1', 'BHPSKI1'),
   ('FNSB-Birch Hill Ski Trail #2', 'BHPSKI2'),
   ('FNSB-Birch Hill Ski Trail #3', 'BHPSKI3'),
   ('FNSB-Birch Hill Ski Trail #4', 'BHPSKI4'),
   ('FNSB-Birch Hill Timing Building', 'BHPBHL'),
   ('FNSB-Birch Hill Warm-up Building', 'BHPBHW'),
   ('FNSB-CLRA Chena Lakes Rec Area', 'CLX001'),
   ('FNSB-CLRA Chena Lakes Rec Park', 'CLX002'),
   ('FNSB-CLRA Chena River Lakes Proj A', 'CLX004'),
   ('FNSB-CLRA Chena River Lakes Rec Proj', 'CLX003'),
   ('FNSB-CLRA Entrance Station', 'CLXES1'),
   ('FNSB-CLRA Maintenance Bunker', 'CLXSM1'),
   ('FNSB-CLRA Office/Shop', 'CLXSO1'),
   ('FNSB-CLRA River Park General Area', 'CLXGP2'),
   ('FNSB-Carlson Center', 'CACBG1')